In [1]:
import pandas as pd
# QA-SRL2
orig_dev = pd.read_json('./data/qasrl-v2/orig/dev.jsonl', lines=True)
# Training set for Supervised_OIE
train = pd.read_csv('./data/train.oie.conll', sep="\t", header = 0)


In [12]:
# Exploratory for QA-SRL
ids = orig_dev['sentenceId']
tokens = orig_dev['sentenceTokens']
entries = orig_dev['verbEntries']
# print(ids[0])
# print(tokens[0])
# print(entries[0]['1']['questionLabels'])

In [5]:
def format_label(label: str):
    if label == 'O':
        return label
    else:
        swapped = []
        swapped.append(label.split("-")[1]) # BIO Tag
        if 'P' in label: # Semantic Role Labeler using B-V instead of B-P
            swapped.append("V")
        else: # SRL BIO-ARGX
            swapped.append(label.split("-")[0][0] + "RG" + label.split("-")[0][1:])
        return "-".join(swapped)
    
sentence_indexes = train.loc[train['word_id'] == 0].index
for i in range(len(sentence_indexes)):
    start_index = sentence_indexes[i]
    end_index = sentence_indexes[i + 1] - 1 if (i + 1) < len(sentence_indexes) else train.last_valid_index()
    sentence = train.loc[start_index : end_index, :]
    pred_ids = sentence.loc[start_index:start_index, "pred_id"].item()
    pred_ids = [int(i) for i in pred_ids[1:-1].split(", ")]
    encoded_preds = [1 if i in pred_ids else 0 for i in range(len(sentence))]
    encoded_pred = sentence["pred_id"].tolist()
    words = sentence["word"].tolist()
    labels = sentence["label"].tolist()
    labels = [format_label(label) for label in labels]
#     print(labels)


In [1]:
import pandas as pd
df = pd.read_csv('./annotated/NeuralRE_2016.txt', sep="\t", names = ['idx1', 'idx2', 'ent1', 'ent2', 'rel', 'sentence', 'end'])

/Users/taigemini/anaconda3/envs/oie/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df = df[lambda x: x['rel'].notnull()] # Filter null
del df["end"]

In [1]:
import sqlite3
import pandas as pd
# Create your connection.
cnx = sqlite3.connect('./data/store.db')
cursor = cnx.cursor()
# print("INSERT INTO {} ({}, {}, {}) VALUES ({}, {}, {})"
#                .format("Sentence", "sentence", "valid_keys", "invalid_keys", "a", "b", "c"))
cursor.execute("INSERT INTO Sentence (sentence, valid_keys, invalid_keys) VALUES (\"1,2,345,44\", 3434, 4445)")
print(cursor.lastrowid)
df = pd.read_sql_query("SELECT * FROM Sentence", cnx)
print(df)
df = pd.read_sql_query("SELECT * FROM PositiveInstance", cnx)
print(df)
df = pd.read_sql_query("SELECT * FROM NegativeInstance", cnx)
print(df)
cnx.close()


1
   sentence_id    sentence  valid_keys  invalid_keys
0            1  1,2,345,44        3434          4445
Empty DataFrame
Columns: [instance_id, entity1, entity2, rel]
Index: []
Empty DataFrame
Columns: [instance_id, entity1, entity2, rel]
Index: []
